## 本篇為nanomaterial application


#### 石墨烯與碳奈米管的結構放置在modula 4的nanomaterial內
#### 小分子的結構則以各自命名放置在modula 4的各自資料夾內

## A 將多個分子組合在同一個系統

### 0.先將要進行merge的分子結構，複製一份至working_dir
### 1. 打開 VMD ( 預設路徑應為: C:\Program Files (x86)\University of Illinois\VMD ; 若是使用Mac則應該是/Users/aa)
### 2. 點VMD Main的【`Extensions`】→【`Tk Console`】→鍵入
###  cd   ~/Desktop/Working_dir/
### 3.打開 JupyterLab → 在左方視窗中移動路徑到 Desktop/NAMD_Tutorial_2022/Working_dir/ 中 → 在右方 Launcher 介面中點 Other 區塊中的 Text File 的選項，以開啟一個新的 txt 檔案。

### → 在 JupyterLab 介面中的untitled.txt視窗中鍵入或(複製貼上下列指令)
    #load nanomaterial
    package require topotools 1.8
    set midlist {}
    set mol [mol new graphene.psf waitfor all]
    mol addfile graphene.pdb

    #load molecule in system
    lappend midlist $mol
set mol [mol new phenol.psf waitfor all]
mol addfile phenol.pdb $mol

    #moving molecule 
    set sel [atomselect top all]
    $sel moveby {0.0 0.0 15.0}
    lappend midlist $mol
    #do the magic
    set mol [::TopoTools::mergemols $midlist]

    animate write psf graphene_sim_temp.psf $mol
animate write pdb graphene_sim_temp.pdb $mol

###  →在左方視窗中以滑鼠"右鍵"點擊名為`untitled.txt` 的檔案，選擇 Rename 並改名為 `merge.tcl`
    
### 4. 在　`Tk Console`　輸入(可以用複製的方式並貼上)：
    source merge.tcl
    
###  →如此會產生兩個結構merge的新檔案：graphene_sim_temp.psf graphene_sim_temp.pdb

## B. merge.tcl 可調整參數之介紹
### 1.merge.tcl 內可拆成四個部分
#### 第一部分：載入奈米材料結構(母體)
    #load nanomaterial
    package require topotools 1.8
    set midlist {}
    set mol [mol new graphene.psf waitfor all]  #這裡的{graphene}.psf可以改任何你想要nanomaterial
    mol addfile graphene.pdb                    #這裡的{graphene}.pdb要跟上一行名稱相同
#### 第二部分：載入小分子藥物
    #load molecule in system
    lappend midlist $mol
set mol [mol new phenol.psf waitfor all]        #這裡的{phenol}.psf可以改任何你想要小分子
mol addfile phenol.pdb $mol                     #這裡的{phenol}.pdb要跟上一行名稱相同
#### 第三部分：移動小分子藥物的位置
    #moving molecule 
    set sel [atomselect top all]
    $sel moveby {0.0 0.0 0.0}                   #大括弧內的三個數值x y z的移動向量
lappend midlist $mol
    #do the magic
    set mol [::TopoTools::mergemols $midlist]
    
#### 第四部份：輸出merge檔案
    animate write psf graphene_sim_temp.psf $mol  #這裡的{graphene_sim_temp}可以改成你想要輸出的名稱
animate write pdb graphene_sim_temp.pdb $mol  #這裡的{graphene_sim_temp}要與上一行相同

### 2.update structure
#### 無論是否有調整，當不再繼續調整後，打開vmd > tkconsole鍵入
source update.tcl
並在下方輸入
graphene_sim 

並按enter即可
#### 即可更新你所調整的分子結構

## C. 設定模擬參數
### C1-1. solvate system in water
### C1-2. 在　`Tk Console`　"逐行"鍵入(可以用"逐行"複製的方式並貼上)：
    package  require  solvate
    solvate  graphene_sim.psf  graphene_sim.pdb -t 5 -o graphene_sim_wb
###  Note→此指令將graphene附近（> 5 angstrom）加入一堆水分子形成graphene水合模型（正立方水盒子），並產生兩個新檔案：
###  graphene_sim_wb.pdb, graphene_sim_wb.psf
### C1-3. VMD Main 中已自動新增一 PSF檔案名為 `graphene_sim_wb.psf` 並讀進了一PDB結構檔案名為 `graphene_sim_wb.pdb` 故 Frames 為 1
### C1-4. 回到VMD　`Tk Console`　鍵入(同樣可以用“逐行”複製的方式並貼上)：
### (注意 'graphene_sim_wb.psf'前面是否有"T"字母)

    set everyone [atomselect top all] 
    measure center $everyone
    
                                >>>>會得一些數值(`center_x` `center_y` `center_z`) 
                                
    measure minmax $everyone

                                >>>>會得一些數值{`min_x` `min_y` `min_z`} {`max_x` `max_y` `max_z`}
                                
###  →上面的指令分別計算整個盒子的 x,y,z “中心” 以及“大小”

### C1-5. 在 JupyterLab 在左方視窗中以滑鼠打開名為:graphene_sim_wb.conf 的檔案
    檢查以下參數 (最開頭):
  
    structure          graphene_sim_wb.psf      (注意路徑)
    coordinates        graphene_sim_wb.pdb      (注意路徑)

    set outputname     graphene_sim_wb_eq

    >根據剛剛在 B. 3 章節中，得到的{min_x min_y min_z} {max_x max_y max_z}
    
    >計算 max_x - min_x 得 length_x
         max_y - min_y 得 length_y
         max_z - min_z 得 length_z

    >將 {length_x length_y length_z} 以及 {center_x center_y center_z} 的值取小數點至後一位，分別填入以下位置
    (已填好，只需檢查是否與預設的值一樣即可)
    
    
    cellBasisVector1  length_x         0.0        0.0
    cellBasisVector2       0.0    length_y        0.0
    cellBasisVector3       0.0         0.0   length_z
    cellOrigin        center_x    center_y   center_z

    請確認是否參數如下：
    cellBasisVector1    65.0    0.0    0.0
    cellBasisVector2     0.0   65.0    0.0
    cellBasisVector3     0.0    0.0   25.0
    cellOrigin          -0.6   -0.7    8.0

### C2-1. vaccum system
### 在 JupyterLab 在左方視窗中以滑鼠打開名為:graphene_sim.conf 的檔案
    檢查以下參數 (最開頭):
  
        structure          graphene_sim.psf      (注意路徑)
        coordinates        graphene_sim.pdb      (注意路徑)

        set outputname     graphene_sim_eq  


    並直接指定PBC 特定x y z的數值即可進到下一步
        cellBasisVector1        75.0         0.0	    0.0
        cellBasisVector2         0.0        75.0        0.0
        cellBasisVector3	     0.0         0.0       60.0
              cellOrigin         0.0         0.0        0.0

## D.開始使用分子動力學模擬程式 (NAMD)
### 1.在 JupyterLab 在左方視窗中點擊 " + " → 在右方 Launcher 介面中點 Other 區塊中的 Terminal → 鍵入(可複製貼上) :
(Mac)
cd ~/Desktop/Working_dir/

(Windows)
cd ~\Desktop\Working_dir\ 
### 再鍵入(可複製貼上) :
(Mac)
~/Desktop/NAMD_Tutorial_2022/NAMD_mac/namd2 +setcpuaffinity +p4 ./graphene_sim_wb.conf > graphene_sim_wb.log

(Windows)
~\Desktop\NAMD_Tutorial_2022\NAMD_exe\namd2 +setcpuaffinity +p4 .\graphene_sim_wb.conf > graphene_sim_wb.log